In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
!pip install pyttsx3
import pyttsx3
text_speech= pyttsx3.init('dummy')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
!pip install SpeechRecognition
import speech_recognition as sr
!tar xzvf pa_stable_v190700_20210406.tgz
!cd pa_stable_v190700_20210406.tgz
!./configure
!make
!make install

!apt-get portaudio19-dev
!apt-get python3-all-dev
!pip install pyaudio
# !apt-get install python3-pyaudio
from IPython.display import HTML, Audio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
portaudio/
portaudio/.editorconfig
portaudio/.gitattributes
portaudio/.github/
portaudio/.github/ISSUE_TEMPLATE/
portaudio/.github/ISSUE_TEMPLATE/bug_report.md
portaudio/.github/workflows/
portaudio/.github/workflows/MSBuild.yml
portaudio/.github/workflows/c-cpp.yml
portaudio/.gitignore
portaudio/CMakeLists.txt
portaudio/Doxyfile
portaudio/Doxyfile.developer
portaudio/LICENSE.txt
portaudio/Makefile.in
portaudio/README.configure.txt
portaudio/README.md
portaudio/SConstruct
portaudio/aclocal.m4
portaudio/bindings/
portaudio/bindings/cpp/
portaudio/bindings/cpp/AUTHORS
portaudio/bindings/cpp/COPYING
portaudio/bindings/cpp/ChangeLog
portaudio/bindings/cpp/INSTALL
portaudio/bindings/cpp/Makefile.am
portaudio/bindings/cpp/Makefile.in
portaudio/bindings/cpp/NEWS
portaudio/bindings/cpp/README
portaudio/bindings/cpp/SConscript
portaudio/bindings/cpp/aclocal.m4
portaudio/bindings/cpp/bin/
portaudio

In [ ]:
lemmatizer = WordNetLemmatizer()
checks = json.loads(open('data.json').read())
material_rates = json.loads(open('material_rate.json').read())

findings = pickle.load(open('queries.pkl','rb'))
category = pickle.load(open('tags.pkl', 'rb'))
model = load_model('Model.h5')


In [ ]:
def final_Sen(line):
    line_findings = nltk.word_tokenize(line)
    line_findings = [lemmatizer.lemmatize(word) for word in line_findings]
    return line_findings

def container_of_findings(line):
    line_findings = final_Sen(line)
    container = [0] * len(findings)
    for w in line_findings:
        for i, point in enumerate(findings):
            if point == w:
                container[i] = 1
    
    return np.array(container)






In [ ]:

def testing(line):
    mem = container_of_findings(line)
    select = model.predict(np.array([mem]))[0]
    thselecth = 0.25
    final = [[i, r] for i , r in enumerate(select) if r > thselecth]

    final.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    for r in final:
        return_list.append({'intent':category[r[0]], 'probability': str(r[1])})
    
    return return_list,category[r[0]]

def choosen_answers(checks_list, checks_json):
    selectult=""
    tag = checks_list[0]['intent']
    list_of_checks = checks_json['intents']
    for i in list_of_checks:
        if i['tag'] == tag:
            selectult = random.choice(i['responses'])
            break
    
    return selectult

def call_follow(class_is):
    num=random.randint(0,1)
    if num==1:
        for intent in checks['intents']:
            if intent['tag']==class_is:
                item = random.choice(intent['followup'])
                call_audio(item)


def call_audio(select):
    print(select)
    text_speech.say(select)
    text_speech.runAndWait()
    
def call_text():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak Customer:")
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio)
            return text
        except:
            print("Sorry could not recognize what you said")
def get_txt_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result
  
def choose_follow(class_is):
    num=random.randint(0,1)
    if num==1:
        for intent in checks['intents']:
            if intent['tag']==class_is:
              item = random.choice(intent['followup'])
              break
        return item

In [ ]:
def get_price_response(material):
    response = ""
    materials = material_rates["material_rate"]
    count = 0
    found = 0
    # call_audio('We sell gold, silver, platinum and diamond. Which material would you like to know about first?')
    # material = call_text()
    # material = input("We sell gold, silver, platinum and diamond. Which material would you like to know about first?")
    while 1:
        if material.find('diamond')!=-1:
            response = 'The current rate for one 4 carat diamond is 329230 rupees per gram, 1/2 carat diamond is 65847 rupees per gram, and 1 carat is 131693 per gram.'
            found =1
        elif material.find("gold")!=-1:
            
            material = "gold"
            response = 'The current rate for 24 carat gold is 140500 rupees per tola, 22 carat gold is 128791 rupees per tola, 21 carat gold is 122938 per tola, and 18 carat gold is 105375 rupees per tola'
            found =1

        elif material.find("silver")!=-1:
            material = "silver"
            response = 'The current rate for silver is 1678 rupees per tola, and 144 rupees per gram'
            found =1

        elif material.find("platinum")!=-1:
            material = "platinum"
            response = 'The current rate for platinum is 72656 rupees per tola, and 6236 rupees per gram'
            found =1
        else:
            found = 0   # no relevant material present
            # call_audio('We sell gold, silver, platinum and diamond. Which material would you like to know about first?')
            # material = call_text()
            material = input("We sell gold, silver, platinum and diamond. Which material would you like to know about first?")
        if found == 1 or material.find("gold")!=-1 or material.find("silver")!=-1 or material.find("platinum")!=-1 or material.find("diamond")!=-1:
            break 
    return response

In [ ]:
import speech_recognition as sr

print("Intelligent AI Chatbot. . . ")

while True:
    # message = call_text()
    message = input("")
    ints,class_is = testing(message)
    # print(message)

    response = choosen_answers(ints, checks)
   
    if(class_is=='goodbye'):
        # call_audio(response)
        print(response)
        break
        
    elif(class_is=='materialpricegivenweight'):
        response=get_price_response(message)
        # call_audio(response)
        print(response)
        follow = choose_follow(class_is)
        if follow is not None:
          print(follow)
    else:
        # call_audio(response)
        print(response)
        # call_follow(class_is)
        follow = choose_follow(class_is)
        if follow is not None:
          print(follow)
   

Intelligent AI Chatbot. . . 
Hello!
Hey! Welcome to our shop. What can we do for you ?
How are you?
Always looking forward to helping our customers !!
Can you help me then?
Any Time
So, then have a look at our best seller :))
What is your best selling item?
As we mostly deal in diamond and gold jewellery, so the rings are our best selling article
Great! What's the price of gold today?
The current rate for 24 carat gold is 140500 rupees per tola, 22 carat gold is 128791 rupees per tola, 21 carat gold is 122938 per tola, and 18 carat gold is 105375 rupees per tola
What would you like to buy?
I want a beautiful gold ring
Goodbye!
